In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import copy

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import matplotlib.pyplot as plt

# from tqdm.notebook import tqdm

color_list = [x["color"] for x in plt.rcParams["axes.prop_cycle"]]

from matplotlib.ticker import MaxNLocator

import multiprocessing
import os

In [ ]:
from lut import *

## MNIST

In [ ]:
def load_mnist(n_samples=10_000, pca=False, n_components=8):
    data = np.load("data/lut/MNIST.npz", allow_pickle=True)
    X_ = data["X"]
    y_ = data["y"]
    
    assert n_samples <= 70_000, f"Full data available only has 70_000 samples"

    if pca:
        pca_ = PCA(n_components=n_components)
        X_ = pca_.fit_transform(X_)

    scaler = MinMaxScaler(feature_range=(0, 1))
    X_tf = scaler.fit_transform(X_)

    X = (X_tf > 0.5).astype(bool)
    y = (y_ == 0) | (y_ == 1) | (y_ == 2) | (y_ == 3) | (y_ == 4)

    X, y = shuffle(X, y, n_samples=n_samples, random_state=100)

    X_train, X_test, y_train, y_test, = train_test_split(
        X, y, test_size=0.2, random_state=42, shuffle=False
    )
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = load_mnist(n_samples=30_000, pca=False, n_components=8)

## Experiments

In [ ]:
# 79 80

In [ ]:
%%time
hidden_layers = [1024] * 10
lut = Lut(
    bits=[2] * (len(hidden_layers) + 1),
    hidden_layers=hidden_layers,
    make_aig=False,
    discard_randoms=False,
    patience=10
)

preds_train = lut.train(X_train, y_train)
preds_test = lut.predict(X_test)
print(
    f"Accuracy on training set (no major): {accuracy_score(preds_train, y_train):.2f}"
)
print(f"Accuracy on test set (no major): {accuracy_score(preds_test, y_test):.2f}")

print()

# preds_train = lut.predict(X_train, majority_vote=True)
# preds_test = lut.predict(X_test, majority_vote=True)
# print(f"Accuracy on training set (major): {accuracy_score(preds_train, y_train):.2f}")
# print(f"Accuracy on test set (major): {accuracy_score(preds_test, y_test):.2f}")

# print()

In [ ]:
frac_rnd = [
    np.any(lut.rnd_arr_[i], axis=1).sum() / lut.rnd_arr_[i].shape[0]
    for i in range(len(lut.rnd_arr_) - 1)
]

frac_not_used = (
    np.array(
        [
            len(
                np.where(
                    np.bincount(
                        np.hstack((np.unique(lut.cols_arr_[i]), list(range(1024))))
                    )
                    != 2
                )[0]
            )
            for i in range(1, len(hidden_layers))
        ]
    )
    / 1024
)
frac_not_used_0 = (
    len(
        np.where(
            np.bincount(np.hstack((np.unique(lut.cols_arr_[0]), list(range(784))))) != 2
        )[0]
    )
    / 784
)
frac_not_used = np.insert(frac_not_used, 0, frac_not_used_0)

fig, axs = plt.subplots(1, 2, figsize=(10, 4))

ax = axs[0]
ax.plot(np.arange(1, len(frac_rnd) + 1), frac_rnd, "o-")
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_title("Fraction random")

ax = axs[1]
ax.plot(np.arange(1, len(frac_rnd) + 1), frac_not_used, "o-")
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_title("Fraction not used")

plt.tight_layout()

## Combining many luts

In [ ]:
hidden_layers = [1024] * 10

preds_train = []
preds_test = []
luts = []

for _ in tqdm(range(10)):
    lut = Lut(
        bits=[2] * (len(hidden_layers) + 1),
        hidden_layers=hidden_layers,
        make_aig=False,
        discard_randoms=False,
        patience=10,
        verbose=False
    )
    luts.append(lut)
    preds_train.append(lut.train(X_train, y_train))
    preds_test.append(lut.predict(X_test))

In [ ]:
train_ = np.array(preds_train).astype(int)
train_[train_ == 0] = -1
train_ = train_.sum(0)

cumul_preds_train = np.zeros_like(train_, dtype=bool)
cumul_preds_train[train_ >= 0] = True

print(f"Accuracy on training set: {accuracy_score(cumul_preds_train, y_train):.2f}")

In [ ]:
test_ = np.array(preds_test).astype(int)
test_[test_ == 0] = -1
test_ = test_.sum(0)

cumul_preds_test = np.zeros_like(test_, dtype=bool)
cumul_preds_test[test_ >= 0] = True

print(f"Accuracy on testing set: {accuracy_score(cumul_preds_test, y_test):.2f}")

## Visualizing data set

In [ ]:
size = 4
idxs = np.random.randint(low=0, high=X_train.shape[0], size=size ** 2)

enlarge = 2
fig, axs = plt.subplots(size, size, figsize=(size * enlarge, size * enlarge))

for idx, ax in enumerate(axs.flatten()):
    ax.imshow(X_train[idxs[idx]].reshape((28, 28)))
    lbl = int(y_train[idxs[idx]])
    if lbl:
        ax.set_title(f"Label 1 (0-4)")
    else:
        ax.set_title(f"Label 0 (5-9)")

plt.tight_layout()

## MLP

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(100,))
clf.fit(X_train, y_train)
preds_mlp_train = clf.predict(X_train)
preds_mlp_test = clf.predict(X_test)

print(f"Accuracy on training set: {accuracy_score(preds_mlp_train, y_train):.2f}%")
print(f"Accuracy on test set: {accuracy_score(preds_mlp_test, y_test):.2f}%")

### Majority Vote

In [ ]:
bit_arr = [4, 5, 6, 7, 8]
majority_arr = [True, False]
acc_train_majority = []
acc_train_no_majority = []
acc_test_majority = []
acc_test_no_majority = []
for bit in bit_arr:
    for majority in majority_arr:
        lut = Lut(bits=[bit] * 4, hidden_layers=[1000] * 3, majority_vote=majority)
        preds_train = lut.train(X_train, y_train)
        preds_test = lut.predict(X_test)

        if majority:
            acc_train_majority.append(accuracy_score(preds_train, y_train))
            acc_test_majority.append(accuracy_score(preds_test, y_test))
        else:
            acc_train_no_majority.append(accuracy_score(preds_train, y_train))
            acc_test_no_majority.append(accuracy_score(preds_test, y_test))

In [ ]:
acc_train_majority, acc_train_no_majority, acc_test_majority, acc_test_no_majority

### Discard bad Luts

In [ ]:
lut = Lut(bits=[8] * 4, hidden_layers=[100] * 3, majority_vote=False)
preds_train = lut.train(X_train, y_train)
preds_test = lut.predict(X_test)

print(f"Accuracy on training set: {accuracy_score(preds_train, y_train):.2f}%")
print(f"Accuracy on test set: {accuracy_score(preds_test, y_test):.2f}%")

acc = lut.get_accuracies_per_layer(X_test, y_test)

In [ ]:
fig, axs = plt.subplots(1, len(acc) - 1, figsize=(9, 3))
for idx, ax in enumerate(axs):
    ax.hist(acc[idx], bins=20)
    xrange = ax.get_xlim()[1] - ax.get_xlim()[0]
    yrange = ax.get_ylim()[1] - ax.get_ylim()[0]
    ax.text(ax.get_xlim()[0] + xrange * 0.1, ax.get_ylim()[0] + yrange * 0.8, f"$\mu=${np.mean(acc[idx]):.2f}")
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.set_title(f"Test accuracies for\nhidden layer {idx + 1},\n{len(acc[idx])} luts")
    ax.set_xlabel("Accuracy")
    ax.set_ylabel("Occurrence")
    
plt.tight_layout()

In [ ]:
bit_arr = [4, 5, 6, 7, 8]
discard_arr = [True, False]
acc_train_discard = []
acc_train_no_discard = []
acc_test_discard = []
acc_test_no_discard = []

for bit in bit_arr:
    tmp_acc_train_discard = []
    tmp_acc_train_no_discard = []
    tmp_acc_test_discard = []
    tmp_acc_test_no_discard = []
    for _ in range(10):
        for discard in discard_arr:
            lut = Lut(
                bits=[bit] * 4,
                hidden_layers=[100] * 3,
                majority_vote=False,
                discard_bad_luts=discard,
                des_acc=[0.63, 0.83, 0.86],
                discard_num = 10,
                patience=10
            )
            preds_train = lut.train(X_train, y_train)
            preds_test = lut.predict(X_test)

            if discard:
                tmp_acc_train_discard.append(accuracy_score(preds_train, y_train))
                tmp_acc_test_discard.append(accuracy_score(preds_test, y_test))
            else:
                tmp_acc_train_no_discard.append(accuracy_score(preds_train, y_train))
                tmp_acc_test_no_discard.append(accuracy_score(preds_test, y_test))

    acc_train_discard.append(tmp_acc_train_discard)
    acc_train_no_discard.append(tmp_acc_train_no_discard)
    acc_test_discard.append(tmp_acc_test_discard)
    acc_test_no_discard.append(tmp_acc_test_no_discard)